<a href="https://colab.research.google.com/github/arvindsuresh-math/Fall-2025-Team-Big-Data/blob/main/notebooks/ablation_additive_axis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Setup and Installations

This cell prepares the Google Colab environment by mounting Google Drive, changing the working directory to our project folder to ensure all custom modules can be imported, installing the required Python packages, and handling Hugging Face authentication.

In [1]:
# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Change Directory to Project Folder ---
# This is a crucial step that makes all local imports work seamlessly
import os
# IMPORTANT: Make sure this path matches the location of your project folder in Google Drive
PROJECT_PATH = '/content/drive/MyDrive/Airbnb_Price_Project'
os.chdir(PROJECT_PATH)
print(f"Current working directory: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory: /content/drive/MyDrive/Airbnb_Price_Project


In [2]:
# --- Hugging Face Authentication ---
from google.colab import userdata
from huggingface_hub import login
print("\nAttempting Hugging Face login...")
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Hugging Face login successful.")
except Exception as e:
    print(f"Could not log in. Please ensure 'HF_TOKEN' is a valid secret. Error: {e}")


Attempting Hugging Face login...
Hugging Face login successful.


In [3]:
# --- Install Dependencies ---
!pip install pandas
!pip install pyarrow
!pip install sentence-transformers
!pip install scikit-learn
!pip install torch
!pip install tqdm
!pip install transformers
!pip install matplotlib
!pip install seaborn

In [4]:
# --- 2. Standard and Third-Party Library Imports ---
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer

# --- 3. Imports from Your Custom Project Scripts ---
print("\n--- Importing Custom Modules ---")
from config import config
from utils import set_seed
from data_processing import load_and_split_data, FeatureProcessor, create_dataloaders
from model import AirbnbPriceDataset # Re-using the original dataset class
from train import train_model, evaluate_model # Re-using the original train/eval functions


--- Importing Custom Modules ---


In [5]:
class AblationAdditiveAxisModelV2(nn.Module):
    """
    A flexible AdditiveAxisModel for advanced ablation studies (V2).

    New Features:
    - Dropout: Includes dropout in all MLP sub-networks for regularization.
    - Trainable Heads: Allows for selectively making the text-based sub-networks
      (amenities, description) trainable or keeping them frozen.
    """
    def __init__(self, processor: 'FeatureProcessor', config: dict,
                 exclude_axes: list = None,
                 trainable_text_heads: list = None,
                 dropout_rate: float = 0.4):
        super().__init__()
        self.config = config
        self.device = self.config['DEVICE']
        self.exclude_axes = set(exclude_axes) if exclude_axes else set()
        self.trainable_text_heads = set(trainable_text_heads) if trainable_text_heads else set()

        print(f"Initializing Model V2 | Excluded: {self.exclude_axes} | Trainable Heads: {self.trainable_text_heads}")

        # --- Embeddings & Transformer ---
        self.embed_property_type = nn.Embedding(len(processor.vocabs['property_type']), 8)
        self.embed_room_type = nn.Embedding(len(processor.vocabs['room_type']), 4)
        self.text_transformer = SentenceTransformer(self.config['TEXT_MODEL_NAME'])
        for param in self.text_transformer.parameters():
            param.requires_grad = False
        # Last layer of transformer is always open for fine-tuning
        for param in self.text_transformer[0].auto_model.encoder.layer[-1].parameters():
            param.requires_grad = True

        # --- Helper to create MLPs with Dropout and BatchNorm ---
        def _create_mlp(in_features, layer_sizes):
            layers = []
            for size in layer_sizes:
                layers.append(nn.Linear(in_features, size))
                layers.append(nn.BatchNorm1d(size))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(p=dropout_rate))
                in_features = size
            # Remove final Dropout before the head
            return nn.Sequential(*layers[:-1])

        # --- Sub-network Bodies ---
        text_embed_dim = self.text_transformer.get_sentence_embedding_dimension()
        self.loc_subnet_body = _create_mlp(self.config['GEO_EMBEDDING_DIM'], self.config['HIDDEN_LAYERS_LOCATION'])
        self.size_subnet_body = _create_mlp(16, self.config['HIDDEN_LAYERS_SIZE_CAPACITY'])
        self.qual_subnet_body = _create_mlp(8, self.config['HIDDEN_LAYERS_QUALITY'])
        self.amenities_subnet_body = _create_mlp(text_embed_dim, self.config['HIDDEN_LAYERS_AMENITIES'])
        # CORRECTED: Changed 'desc_subnet_body' to 'description_subnet_body'
        self.description_subnet_body = _create_mlp(text_embed_dim, self.config['HIDDEN_LAYERS_DESCRIPTION'])
        self.season_subnet_body = _create_mlp(2, self.config['HIDDEN_LAYERS_SEASONALITY'])

        # --- Sub-network Heads ---
        self.loc_subnet_head = nn.Linear(self.config['HIDDEN_LAYERS_LOCATION'][-1], 1)
        self.size_subnet_head = nn.Linear(self.config['HIDDEN_LAYERS_SIZE_CAPACITY'][-1], 1)
        self.qual_subnet_head = nn.Linear(self.config['HIDDEN_LAYERS_QUALITY'][-1], 1)
        self.amenities_subnet_head = nn.Linear(self.config['HIDDEN_LAYERS_AMENITIES'][-1], 1)
        # CORRECTED: Changed 'desc_subnet_head' to 'description_subnet_head'
        self.description_subnet_head = nn.Linear(self.config['HIDDEN_LAYERS_DESCRIPTION'][-1], 1)
        self.season_subnet_head = nn.Linear(self.config['HIDDEN_LAYERS_SEASONALITY'][-1], 1)

        # --- FREEZE/UN-FREEZE LOGIC FOR TEXT HEADS ---
        # This loop will now work correctly with the updated attribute names
        for axis_name in ['amenities', 'description']:
            is_trainable = axis_name in self.trainable_text_heads
            body_net = getattr(self, f'{axis_name}_subnet_body')
            head_net = getattr(self, f'{axis_name}_subnet_head')
            for param in body_net.parameters():
                param.requires_grad = is_trainable
            for param in head_net.parameters():
                param.requires_grad = is_trainable

        self.to(self.device)

    def forward(self, batch: dict) -> torch.Tensor:
        # --- Prepare Inputs ---
        loc_input = batch['loc_geo_position']
        size_input = torch.cat([
            self.embed_property_type(batch['size_property_type']), self.embed_room_type(batch['size_room_type']),
            batch['size_accommodates'].unsqueeze(1), batch['size_bedrooms'].unsqueeze(1),
            batch['size_beds'].unsqueeze(1), batch['size_bathrooms'].unsqueeze(1)
        ], dim=1)
        qual_cols = [
            "review_scores_rating", "review_scores_cleanliness", "review_scores_checkin",
            "review_scores_communication", "review_scores_location", "review_scores_value",
            "total_reviews", "host_is_superhost"
        ]
        qual_input = torch.cat([batch[f'qual_{c}'].unsqueeze(1) for c in qual_cols], dim=1)
        amenities_tokens = {k: v.squeeze(1) for k, v in batch['amenities_tokens'].items()}
        desc_tokens = {k: v.squeeze(1) for k, v in batch['description_tokens'].items()}
        amenities_embed = self.text_transformer(amenities_tokens)['sentence_embedding']
        desc_embed = self.text_transformer(desc_tokens)['sentence_embedding']

        # --- Get Contributions ---
        p_loc = self.loc_subnet_head(self.loc_subnet_body(loc_input))
        p_size = self.size_subnet_head(self.size_subnet_body(size_input))
        p_qual = self.qual_subnet_head(self.qual_subnet_body(qual_input))
        p_amenities = self.amenities_subnet_head(self.amenities_subnet_body(amenities_embed))
        # CORRECTED: Using the updated attribute name here
        p_desc = self.description_subnet_head(self.description_subnet_body(desc_embed))
        p_season = self.season_subnet_head(self.season_subnet_body(batch['season_cyclical']))

        # --- Ablation Logic ---
        contributions = {
            'location': p_loc, 'size_capacity': p_size, 'quality': p_qual,
            'amenities': p_amenities, 'description': p_desc, 'seasonality': p_season
        }

        predicted_log_deviation = torch.zeros_like(p_loc)
        for axis_name, contribution in contributions.items():
            if axis_name not in self.exclude_axes:
                predicted_log_deviation += contribution

        return predicted_log_deviation.squeeze(-1)

In [6]:
# ==============================================================================
# --- MAIN EXECUTION SCRIPT ---
# ==============================================================================

# 1. Imports and Global Configuration

print("\n--- Applying Global Settings ---")
# Set the random seed for reproducibility
set_seed(config['SEED'])

# Print configuration to verify settings
print("\nConfiguration Settings:")
for key, value in config.items():
    print(f"{key}: {value}")

# Confirm the device
print(f"\nUsing device: {config['DEVICE']}")
if config['DEVICE'] == 'cuda':
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

print("\nSetup and imports complete. Ready to proceed with the ablation study.")



--- Applying Global Settings ---
All random seeds set to 42.

Configuration Settings:
CITY: toronto
DEVICE: cuda
DRIVE_SAVE_PATH: /content/drive/MyDrive/Colab_Notebooks/Airbnb_Project/artifacts/
TEXT_MODEL_NAME: BAAI/bge-small-en-v1.5
VAL_SIZE: 0.05
SEED: 42
BATCH_SIZE: 256
VALIDATION_BATCH_SIZE: 512
LEARNING_RATE: 0.001
TRANSFORMER_LEARNING_RATE: 1e-05
N_EPOCHS: 100
HIDDEN_LAYERS_LOCATION: [32, 16]
HIDDEN_LAYERS_SIZE_CAPACITY: [32, 16]
HIDDEN_LAYERS_QUALITY: [32, 16]
HIDDEN_LAYERS_AMENITIES: [64, 32]
HIDDEN_LAYERS_DESCRIPTION: [64, 32]
HIDDEN_LAYERS_SEASONALITY: [16]
GEO_EMBEDDING_DIM: 32
EARLY_STOPPING_PATIENCE: 10
EARLY_STOPPING_MIN_DELTA: 0.001
SCHEDULER_PATIENCE: 2
SCHEDULER_FACTOR: 0.5

Using device: cuda
GPU Name: NVIDIA A100-SXM4-40GB

Setup and imports complete. Ready to proceed with the ablation study.


In [7]:
# 2. Load and Split Data
train_df, val_df, neighborhood_log_means, _, _ = load_and_split_data(config)
print(f"\nTraining DataFrame shape: {train_df.shape}")
print(f"Validation DataFrame shape: {val_df.shape}")

Loading dataset from: ./toronto_dataset_oct_20.parquet
Stratified split complete. Listings in Train: 7,618, Val: 401
Total records in Train: 82,065, Val: 4,327

Training DataFrame shape: (82065, 29)
Validation DataFrame shape: (4327, 29)


In [8]:
# 3. Process Features
processor = FeatureProcessor(config)
processor.fit(train_df)
train_features = processor.transform(train_df, neighborhood_log_means)
val_features = processor.transform(val_df, neighborhood_log_means)
train_loader, val_loader = create_dataloaders(train_features, val_features, config)

Fitting FeatureProcessor...
Fit complete.


/content/drive/MyDrive/Airbnb_Price_Project/data_processing.py:151: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['neighborhood_log_mean'].fillna(global_mean, inplace=True)
/content/drive/MyDrive/Airbnb_Price_Project/data_processing.py:151: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

DataLoaders created.


In [9]:
# ==============================================================================
# WRAPPER FUNCTION TO RUN AN EXPERIMENT
# ==============================================================================

def run_additive_ablation_experiment_v2(
    config: dict,
    processor: 'FeatureProcessor',
    train_loader: DataLoader,
    val_loader: DataLoader,
    exclude_axes: list = None,
    trainable_text_heads: list = None
):
    """
    Initializes, trains, and evaluates an AblationAdditiveAxisModelV2.
    """
    experiment_desc = f"Exclude: {exclude_axes}, Trainable: {trainable_text_heads}"
    print("\n" + "="*70)
    print(f"  STARTING ADDITIVE ABLATION | {experiment_desc}")
    print("="*70)

    # 1. Instantiate the new V2 model with all options
    model = AblationAdditiveAxisModelV2(
        processor, config,
        exclude_axes=exclude_axes,
        trainable_text_heads=trainable_text_heads
    )

    # 2. Define the optimizer with differential LRs and WEIGHT DECAY
    transformer_params = [p for p in model.text_transformer.parameters() if p.requires_grad]

    # All other trainable parameters
    other_params = [
        p for n, p in model.named_parameters()
        if 'text_transformer' not in n and p.requires_grad
    ]

    optimizer = optim.AdamW([
        {'params': other_params, 'lr': config['LEARNING_RATE'], 'weight_decay': 1e-4},
        {'params': transformer_params, 'lr': config['TRANSFORMER_LEARNING_RATE']}
    ])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=config['SCHEDULER_FACTOR'],
        patience=config['SCHEDULER_PATIENCE']
    )

    # 3. Train the model
    trained_model, _ = train_model(
        model=model, train_loader=train_loader, val_loader=val_loader,
        optimizer=optimizer, scheduler=scheduler, config=config
    )

    # 4. Perform final evaluation
    print(f"\n--- Final Evaluation for | {experiment_desc} ---")
    final_train_mse, final_train_mape = evaluate_model(trained_model, train_loader, config['DEVICE'])
    final_val_mse, final_val_mape = evaluate_model(trained_model, val_loader, config['DEVICE'])

    metrics = {
        "excluded_axes": str(exclude_axes or "[]"),
        "trainable_text_heads": str(trainable_text_heads or "[]"),
        "train_rmse": np.sqrt(final_train_mse), "val_rmse": np.sqrt(final_val_mse),
        "train_mape": final_train_mape, "val_mape": final_val_mape
    }

    print("\n" + "*"*70)
    print(f"  RUN COMPLETE | {experiment_desc}")
    print(f"  Final Validation RMSE: {metrics['val_rmse']:.4f}")
    print(f"  Final Validation MAPE: {metrics['val_mape'] * 100:.3f}%")
    print("*"*70 + "\n")

    return metrics

In [10]:
# ==============================================================================
# 1. RUN THE FULL SUITE OF ABLATION EXPERIMENTS
# ==============================================================================
ablation_results = []

# --- BASELINE: All axes used, both text heads are trainable ---
ablation_results.append(run_additive_ablation_experiment_v2(
    config=config, processor=processor, train_loader=train_loader, val_loader=val_loader,
    exclude_axes=[],
    trainable_text_heads=['amenities', 'description']
))

# --- FROZEN EXPERIMENTS ---
# Both text heads used but FROZEN
ablation_results.append(run_additive_ablation_experiment_v2(
    config=config, processor=processor, train_loader=train_loader, val_loader=val_loader,
    exclude_axes=[],
    trainable_text_heads=[]
))
# Amenities only, FROZEN
ablation_results.append(run_additive_ablation_experiment_v2(
    config=config, processor=processor, train_loader=train_loader, val_loader=val_loader,
    exclude_axes=['description'],
    trainable_text_heads=[]
))
# Description only, FROZEN
ablation_results.append(run_additive_ablation_experiment_v2(
    config=config, processor=processor, train_loader=train_loader, val_loader=val_loader,
    exclude_axes=['amenities'],
    trainable_text_heads=[]
))

# --- TRAINABLE EXPERIMENTS ---
# Amenities only, TRAINABLE
ablation_results.append(run_additive_ablation_experiment_v2(
    config=config, processor=processor, train_loader=train_loader, val_loader=val_loader,
    exclude_axes=['description'],
    trainable_text_heads=['amenities']
))
# Description only, TRAINABLE
ablation_results.append(run_additive_ablation_experiment_v2(
    config=config, processor=processor, train_loader=train_loader, val_loader=val_loader,
    exclude_axes=['amenities'],
    trainable_text_heads=['description']
))

# --- NO TEXT AXES EXPERIMENT ---
# Both text axes completely removed
ablation_results.append(run_additive_ablation_experiment_v2(
    config=config, processor=processor, train_loader=train_loader, val_loader=val_loader,
    exclude_axes=['amenities', 'description'],
    trainable_text_heads=[]
))


  STARTING ADDITIVE ABLATION | Exclude: [], Trainable: ['amenities', 'description']
Initializing Model V2 | Excluded: set() | Trainable Heads: {'description', 'amenities'}

--- Starting Model Training ---
Epoch |     Time |   Train RMSE |   Val RMSE | Val MAPE (%) | Patience
----------------------------------------------------------------------


Epoch 1/100:   0%|          | 0/321 [00:00<?, ?it/s]

Validating:   0%|          | 0/9 [00:00<?, ?it/s]

    1 | 00:01:17 |       0.4677 |     0.3552 |        30.18 |        0


Epoch 2/100:   0%|          | 0/321 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/9 [00:00<?, ?it/s]

   14 | 00:18:29 |       0.3401 |     0.3478 |        29.56 |       10
--- Early Stopping Triggered (MAPE did not improve for 10 epochs) ---

--- Training Complete ---
Loading best model state with Val MAPE: 28.69%

--- Final Evaluation for | Exclude: ['amenities', 'description'], Trainable: [] ---


Validating:   0%|          | 0/321 [00:00<?, ?it/s]

Validating:   0%|          | 0/9 [00:00<?, ?it/s]


**********************************************************************
  RUN COMPLETE | Exclude: ['amenities', 'description'], Trainable: []
  Final Validation RMSE: 0.3478
  Final Validation MAPE: 29.561%
**********************************************************************



In [11]:
# ==============================================================================
# 2. CREATE, DISPLAY, AND SAVE THE FINAL SUMMARY
# ==============================================================================
print("\n\n" + "="*100)
print(f"{'COMPREHENSIVE ADDITIVE ABLATION STUDY SUMMARY':^100}")
print("="*100)

results_df = pd.DataFrame(ablation_results)
results_df['train_mape_pct'] = results_df['train_mape'] * 100
results_df['val_mape_pct'] = results_df['val_mape'] * 100

display_cols = ['excluded_axes', 'trainable_text_heads', 'train_rmse', 'val_rmse', 'train_mape_pct', 'val_mape_pct']
print(results_df[display_cols].to_string(index=False, float_format="%.4f"))
print("="*100)

# Save the results
timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
filename = f"{config['CITY']}_additive_ablation_v2_results_{timestamp}.csv"
save_path = os.path.join(config['DRIVE_SAVE_PATH'], filename)
os.makedirs(config['DRIVE_SAVE_PATH'], exist_ok=True)
results_df.to_csv(save_path, index=False, float_format="%.6f")

print(f"\nComprehensive ablation results successfully saved to:\n{save_path}")



                           COMPREHENSIVE ADDITIVE ABLATION STUDY SUMMARY                            
               excluded_axes         trainable_text_heads  train_rmse  val_rmse  train_mape_pct  val_mape_pct
                          [] ['amenities', 'description']      0.2230    0.3376         17.5890       27.5559
                          []                           []      0.3191    0.3500         25.3470       26.9958
             ['description']                           []      0.3267    0.3488         26.7519       28.0609
               ['amenities']                           []      0.3236    0.3546         27.1943       28.9958
             ['description']                ['amenities']      0.2842    0.3448         23.7507       28.6241
               ['amenities']              ['description']      0.2499    0.3532         20.1409       28.9759
['amenities', 'description']                           []      0.3336    0.3478         29.0042       29.5608

Comprehensive ab